In [53]:
import os
import cv2
import random
import json
import numpy as np
from tabulate import tabulate

class Constants:
  POINTS_QUANTITY = 230 # ? Cantidad de puntos a generar
  REGION_VALUE = 4 # ? Valor que resta a los margenes de la imagen para obtener el area de interes
  IMG_SIZE = 27 # ? Se maneja solo un valor ya que son imagenes cuadradas de la forma LxL px


class LernMatrix(Constants):

  def __init__(self, data_path, sample_path, imgQuantity = 100):
    self.data_path = data_path
    self.sample_path = sample_path
    self.aImgFolders = os.listdir(data_path) if data_path else None
    self.imgQuantity = imgQuantity

# ? --------------------------------------------------------------------------------------------------------------------------------------------------

  def genPointsConfig(self):
    """
    * Funcion que genera la configuracion de los puntos sobre la imagen
    * @return: Lista con las configuraciones de los puntos
    """
    return [
      [
        int(random.uniform(0 + self.REGION_VALUE, self.IMG_SIZE - 2)),
        int(random.uniform(0 + self.REGION_VALUE, self.IMG_SIZE - self.REGION_VALUE))
      ] 
      for _ in range(self.POINTS_QUANTITY)
    ]

# ? --------------------------------------------------------------------------------------------------------------------------------------------------

  def imageBinarization(self, img, pointsConfig):
    """
    * Funcion que binariza la imagen
    * @param img: Ruta de la imagen a binarizar
    * @param pointsConfig: Configuracion de los puntos sobre la imagen
    * @return: Imagen binarizada
    """

    # ? Se inicializa la lista que representará la imagen en formato binario
    binaryIMG = np.zeros((self.POINTS_QUANTITY, ), dtype=int)
    openedImage = cv2.imread(img)

    # ? Referencia => https://stackoverflow.com/questions/9780632/how-do-i-determine-if-a-color-is-closer-to-white-or-black#:~:text=The%20former%20is%20easy%3A%20convert,closer%20to%20white%20(255).
    for index, points in enumerate(pointsConfig):
      (B, G, R) = openedImage[points[0], points[1]]
      luminance = .2126*R + 0.7152*G + 0.0722*B
      
      if luminance >= 128:
        binaryIMG[index] = 1

    return binaryIMG

# ? --------------------------------------------------------------------------------------------------------------------------------------------------

  def readImages(self, folder, pointsConfig):
    """
    * Funcion que lee las imagenes del dataset y dependiendo de la cantidad de imagenes que se desea, retorna una lista con la longitud establecida
    * @param folder: Ruta del folder donde se encuentran las imagenes del dataset
    * @param pointsConfig: Configuracion de los puntos sobre la imagen
    * @return: Lista con las imagenes del dataset
    """
    images = []
    imgCount = 0
    folderPath = f'{self.data_path}/{folder}'
    for filename in os.listdir(folderPath):
        if imgCount < self.imgQuantity:
            imgCount += 1
            images.append(self.imageBinarization(f'{folderPath}/{filename}', pointsConfig))
        else:
          break
    return images

# ? --------------------------------------------------------------------------------------------------------------------------------------------------

  def genImageMatrix(self, pointsConfig):
    """
    * Funcion que genera la matriz de imagenes binarizadas
    * @return: Matriz de imagenes binarizadas
    """
    return list(map(lambda folder: self.readImages(folder, pointsConfig), self.aImgFolders))

# ? --------------------------------------------------------------------------------------------------------------------------------------------------

  def learningPhase(self, imgMatrix, AM):
    """
    * Funcion que trabaja sobre la memoria asociativa a partir de las imagenes del dataset
    * @param imgMatrix: Lista con las imagenes del dataset
    * @param AM: Memoria asociativa
    * @return AM: Memoria asociativa actualizada
    """

    for index, imgList in enumerate(imgMatrix):
      for imgVector in imgList:
        AM[index] = AM[index] + (2 * np.array(imgVector) - 1)

    return AM

# ? --------------------------------------------------------------------------------------------------------------------------------------------------

  def tabulateData(self, data):
    """
    * Funcion que genera una tabla con los datos pasados por parametro
    * @param data: Datos a mostrar en la tabla
    * @return: Tabla con los datos
    """
    print(tabulate(data, headers=['Número esperado', 'Número predicho'], tablefmt='orgtbl'))
    

# ? --------------------------------------------------------------------------------------------------------------------------------------------------

  def testEfficiencyRate(self, AM, pointsConfig):
    """
    * Funcion que calcula la tasa de eficiencia del modelo
    * @param AM: Memoria asociativa
    * @param pointsConfig: Configuracion de los puntos usada en el dataset
    * @return: efficiencyRate: Porcentaje de efectividad del entrenamiento
    """
    files = os.listdir(f'{self.sample_path}')
    correctImgsNum = 0 # * Contador para conocer cuantas imagenes se clasificaron correctamente
    TEST_IMG_QUANTITY = len(files) # * Cantidad de imagenes a clasificar
    JSON_CORRECT_IMG = open("correctNumbers.json") # * Archivo JSON que contiene los números de las imagenes correctamente clasificadas

    correctImgs = json.load(JSON_CORRECT_IMG)["sample-numbers"] 

    for index, _ in enumerate(files):
      imgTest = np.array(self.imageBinarization(f'{self.sample_path}/img_{index + 1}.jpg', pointsConfig))

      x = np.transpose([imgTest])

      # ? Referencia: https://www.statology.org/operands-could-not-be-broadcast-together-with-shapes/
      y = np.transpose(AM.dot(x))[0]

      estimatedNum = np.where(y == np.amax(y))[0][0]
      correctNum = correctImgs[index]

      self.tabulateData([[correctNum, estimatedNum]])

      if (estimatedNum == correctNum):
        correctImgsNum += 1

    efficiencyRate = correctImgsNum / TEST_IMG_QUANTITY

    return efficiencyRate

# ? --------------------------------------------------------------------------------------------------------------------------------------------------

  def fit(self):
    # * Generando la configuracion de los puntos sobre la imagen
    pointsConfig = self.genPointsConfig()
    # print(pointsConfig)

    # * Generando la matriz de imagenes binarizadas
    imageMatrix = self.genImageMatrix(pointsConfig)

    # * Inicializando la matriz de aprendizaje (memoria asociativa)
    AM = np.zeros((len(imageMatrix), self.POINTS_QUANTITY), dtype=int)

    # * Entrenando la memoria asociativa
    AM = self.learningPhase(imageMatrix, AM)

    # * Calculando la tasa de eficiencia del modelo
    efficiencyRate = self.testEfficiencyRate(AM, pointsConfig)

    return round((efficiencyRate * 100), 4)



In [54]:
lms = LernMatrix('dataset/trainingSet', 'dataset/testSample')

lms.fit()



|   Número esperado |   Número predicho |
|-------------------+-------------------|
|                 2 |                 2 |
|   Número esperado |   Número predicho |
|-------------------+-------------------|
|                 0 |                 0 |
|   Número esperado |   Número predicho |
|-------------------+-------------------|
|                 9 |                 8 |
|   Número esperado |   Número predicho |
|-------------------+-------------------|
|                 0 |                 6 |
|   Número esperado |   Número predicho |
|-------------------+-------------------|
|                 3 |                 3 |
|   Número esperado |   Número predicho |
|-------------------+-------------------|
|                 7 |                 7 |
|   Número esperado |   Número predicho |
|-------------------+-------------------|
|                 0 |                 0 |
|   Número esperado |   Número predicho |
|-------------------+-------------------|
|                 3 |             

64.0